In [ ]:
import numpy as np
from viz import quickplot
import matplotlib.pyplot as plt
import mne
import pickle as pkl
from sim import *
from inverse_solutions import *
from source_covs import *
from util import *

## Load data

In [ ]:
with open('assets/leadfield.pkl', 'rb') as file:
    leadfield = pkl.load(file)[0]
with open('assets/pos.pkl', 'rb') as file:  
    pos = pkl.load(file)[0]

## Generate random brain activity

In [ ]:
settings = {"n_sources": 1,  # number of sources
            "diam": 40,  # diameter of source patches in mm
            "amplitude": 1  # amplitude of source patches
            }
x, y = simulate_source(leadfield, pos, settings)

## Plot True Source

In [ ]:
%matplotlib qt
quickplot(y, 'assets', backend='mayavi', title='True Source')

## Exhaustive Dipole Search

In [ ]:
# y_est = exhaustive_dipole_search(x, leadfield, pos)
# quickplot(y_est, 'assets', backend='mayavi')

## Linear least squares (same as MNE)

In [ ]:
# y_least_squares = np.linalg.lstsq(leadfield, x)[0]
# quickplot(y_least_squares, 'assets', backend='mayavi')

## With Source covariance matrix

In [ ]:
# Lets have a little noise
sensorNoise = x * rms(x) * 0.05

## MNE

In [ ]:
%matplotlib qt
y_mne = minimum_norm_estimate(x, leadfield, sensorNoise=sensorNoise)
quickplot(y_mne, 'assets/', backend='mayavi', title='MNE')

## LORETA

## Dipfit

In [ ]:
sensorNoise = x * rms(x) * 0.05
sourceCov = np.zeros((leadfield.shape[1], leadfield.shape[1]))
sourceCov[100, 100] = 1
w = np.matmul( np.matmul(sourceCov, leadfield.T), (np.linalg.inv(sensorNoise + np.matmul(np.matmul(leadfield, sourceCov), leadfield.T))) )
y_est = np.sum(w*x, axis=1)
quickplot(y_est, 'assets/', backend='mayavi', title='LORETA')

## Regularized MNE

In [ ]:
y_est = minimum_norm_estimate_2(x, leadfield, reg=True, alpha=2e-5)
quickplot(y_est, 'assets/', backend='mayavi', title=f'Regularized MNE')

In [ ]:
y_est = minimum_norm_estimate_2(x, leadfield, reg=False)
quickplot(y_est, 'assets/', backend='mayavi', title=f'Unregularized MNE')

In [ ]:
alpha = 2e-5

y_est = minimum_norm_estimate_2(x, leadfield, reg=True, alpha=alpha)
quickplot(y_est, 'assets/', backend='mayavi', title=f'Regularized MNE, alpha={alpha}')

In [ ]:
alpha = 2e-10

y_est = minimum_norm_estimate_2(x, leadfield, reg=True, alpha=alpha)
quickplot(y_est, 'assets/', backend='mayavi', title=f'Regularized MNE, alpha={alpha}')

In [ ]:
%matplotlib qt
y_loreta = loreta(x, leadfield, sigma=10, sensorNoise=sensorNoise)
quickplot(y_loreta, 'assets/', backend='mayavi', title='LORETA')